In [1]:
import pandas as pd
import numpy as np
from causalnex.structure import StructureModel

In [2]:
from causalnex.plots import plot_structure
from causalnex.network import BayesianNetwork

/home/getachew/Documents/10_Academy/Week_8/Logistics_Optimization_with_Casual_Inference/causal_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from sklearn.model_selection import train_test_split
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE
import matplotlib.pyplot as plt
from causalnex.structure.notears import from_pandas
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [6]:
df = pd.read_csv('../data/features.csv')
df.head(3)

,id,order_id,driver_id,driver_action,lat,lng,driver_location,trip_origin,trip_destination,trip_Start_time,trip_end_time,trip_distance,trip_to_driver_distance,trip_time,trip_speed,isHoliday,isWeekend
0,1,392001,243828,accepted,6.602207,3.270465,"6.6022066,3.2704649","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,0.694264,0 days 00:03:37,0.096702,0,0
1,2,392001,243588,rejected,6.592097,3.287445,"6.5920972,3.2874447","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,1.551694,0 days 00:03:37,0.096702,0,0
2,3,392001,243830,rejected,6.596133,3.281784,"6.5961334,3.2817841","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,0.786777,0 days 00:03:37,0.096702,0,0


In [7]:
label_encoder = LabelEncoder()
df['driver_action'] = label_encoder.fit_transform(df['driver_action'])
df['isWeekend'] = label_encoder.fit_transform(df['isWeekend'])

In [8]:
scaler = MinMaxScaler()
scaled_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
train_df, holdout_df = train_test_split(df, test_size=0.2, random_state=42)
scaled_train_df = pd.DataFrame(scaler.fit_transform(train_df), columns=train_df.columns)
scaled_holdout_df = pd.DataFrame(scaler.transform(holdout_df), columns=holdout_df.columns)

ValueError: could not convert string to float: '6.6022066,3.2704649'